In [1]:
import pandas as pd
import numpy as np

In [3]:
# make entire query asin pair all in training or testing to do ranked precision/recall
data_path="data/"
data_name="gold_in_qa_split"

In [5]:
train=pd.read_csv(data_path+data_name+"_train.tsv",sep="\t")
test=pd.read_csv(data_path+data_name+"_test.tsv",sep="\t")

In [8]:
all_data=pd.concat([train,test],ignore_index=True)

In [16]:
queries=all_data["keyword"].unique()

In [35]:
np.random.seed(1)
test_queries=set(np.random.choice(queries,replace=False,size=len(queries)//10))

In [49]:
test_data=all_data[[query in test_queries for query in all_data["keyword"]]]

In [50]:
train_data=all_data[[query not in test_queries for query in all_data["keyword"]]]

In [54]:
test_data.to_csv(data_path+data_name+"_test.tsv",sep="\t",index=False)
train_data.to_csv(data_path+data_name+"_train.tsv",sep="\t",index=False)